In [16]:
!pip install -q -U google-generativeai
!pip install ipywidgets
!pip install numpy
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [18]:
import google.generativeai as genai
import numpy as np
import pandas as pd
GOOGLE_API_KEY ="COLE SUA API KEY AQUI"
genai.configure(api_key=GOOGLE_API_KEY)

Listar modelos

In [19]:
for m in genai.list_models():
    if 'embedContent' in m.supported_generation_methods:
        print(m.name)

models/embedding-001
models/text-embedding-004


Embedding: forma de representar palavras(token) em forma vetorial, verifica a palavra por meio do contexto passado (contexto semântico)

Obs: Podem ser usados documentos, fotos, planilhas e dentre outros.

In [20]:
title = "A próxima geração de IA para desenvolvedores e Google Workspace"
sample_text = ("Titulo: A próxima geração de IA para desenvolvedores e Google Workspace"
               "\n"
               "Artigo completo:\n"
               "\n"
               "Gemini API & Goggle AI Studio: Uma maneira acessível de explorar e criar protótipos com aplicações de IA generativa")

embeddings = genai.embed_content(model="models/embedding-001", content=sample_text, title=title,task_type="RETRIEVAL_DOCUMENT") #gerar embeddings do texto
#task type = tipos que voce irá trabalhar (documentos, querys)
print(embeddings) #retorna a lista de "numeros que formam a apartir do sample_text"


{'embedding': [0.05299908, -0.03557685, -0.021441858, 0.015086849, 0.06556389, 0.0051465286, -0.023320386, -0.016274557, 0.05680594, 0.061549723, 0.009252364, 0.009934231, -0.04727567, -0.021084065, 0.01178617, -0.024537204, 0.01793867, -0.015928214, -0.037124977, -0.0034861693, 0.0050178855, 0.009632896, -0.032720674, -0.067634225, -0.019214673, 0.021491304, 0.012814591, -0.036724273, -0.034163255, 0.023754718, -0.04874855, 0.045606572, -0.03600286, 0.006140492, -0.036286406, -0.04040737, -0.028890518, -0.04911146, -0.003747698, -0.002111273, 0.0031373238, -0.08609798, -0.007603586, 0.027026648, -0.008727415, -0.021800807, 0.047823187, 0.043324456, 0.012378777, -0.056832593, 0.030368453, 0.020549295, 0.06808035, -0.04291779, -0.006365839, -0.008294464, 0.024091918, -0.035606902, 0.022364182, 0.00052945246, -0.0016637429, 0.020267403, -0.022209061, 0.05253558, 0.02228756, -0.054855347, -0.040515613, 0.00051979965, 0.013952545, 0.05217235, 0.0044490504, 0.010418808, 0.06116577, -0.06139

In [21]:
DOCUMENT1 = {
    "title": "Operating the Climate Control System",
    "content": "Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."}
DOCUMENT2 = {
    "title": "Touchscreen",
    "content": "Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the \"Navigation\" icon to get directions to your destination or touch the \"Music\" icon to play your favorite songs."}
DOCUMENT3 = {
    "title": "Shifting Gears",
    "content": "Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [22]:
df = pd.DataFrame(documents) #realiza uma conversão para acessarmos
df.columns = ["Titulo", "Conteudo"]
df

,Titulo,Conteudo
0,Operating the Climate Control System,Your Googlecar has a climate control system th...
1,Touchscreen,Your Googlecar has a large touchscreen display...
2,Shifting Gears,Your Googlecar has an automatic transmission. ...


In [23]:
model = "models/embedding-001"

Criando a função de embedding

In [24]:
def embed_fn(title, text):
    return genai.embed_content(model=model, content=text, title=title,task_type="RETRIEVAL_DOCUMENT")["embedding"] #gerar embeddings do texto e cria uma nova coluna no nosso DataFrame


In [25]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1) #aplica a criação de uma nova coluna
df

,Titulo,Conteudo,Embeddings
0,Operating the Climate Control System,Your Googlecar has a climate control system th...,"[-0.033361107, -0.021217084, -0.049581926, -0...."
1,Touchscreen,Your Googlecar has a large touchscreen display...,"[0.009660736, -0.030662702, -0.017281422, -0.0..."
2,Shifting Gears,Your Googlecar has an automatic transmission. ...,"[-0.04270796, -0.007160868, -0.03242516, -0.02..."


In [28]:
def gerar_e_buscar_consulta(consulta, base, model):#buscar o documento
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)#multiplica o embedding da coluna com a consulta (aquele mais proximo de distância gera um contexto mais próximo)
    #argumento máximo (pega a maior singularidade de contexto)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteudo"]#retorna o texto da coluna conteudo que possui este argumento máximo, ou seja, maior contexto buscado pela conta acima

In [30]:
consulta = "Como faço para trocar marchas em um carro do Google?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.


In [34]:
generation_config = {
  "temperature": 0.5,
  "candidate_count": 1
}

In [35]:
prompt = f"Reescreva esse texto de uma forma mais descontraída, sem adicionar informações que não façam parte do texto: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro", generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

Seu Googlecar tem uma telona sensível ao toque irada que te dá acesso a um monte de coisas bacanas, tipo GPS, música e até pra controlar o ar-condicionado. Pra usar a tela, é só tocar no ícone que você quer. Por exemplo, toca no "GPS" pra saber como chegar no lugar ou no "Música" pra tocar aquela sonzera que você curte.
